In [1]:
library(data.table)

In [104]:
dt <- fread('out.B.tsv')

In [105]:
head(dt)

,V1,V2,V3,V4,V5,V6,V7
1,H5JCKCCXX:2:6:1932827:0/1,B*35:52,245,47,47,1,EDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLPK
2,H5JCKCCXX:2:6:1932827:0/1,B*13:67,245,47,47,1,EDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLPK
3,H5JCKCCXX:2:6:1932827:0/1,B*15:163,245,47,47,1,EDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLPK
4,H5JCKCCXX:2:6:1932827:0/1,B*08:132,245,47,47,1,EDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLPK
5,H5JCKCCXX:2:6:1932827:0/1,B*58:70,245,47,47,1,EDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLPK
6,H5JCKCCXX:2:6:1932827:0/1,B*51:01,245,47,47,1,EDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLPK


In [106]:
setnames(dt, c('q', 't', 'pos', 'len', 'align', 'specific', 'seq'))

In [107]:
nrow(dt)

[1] 68124

In [108]:
tt <- dt[, qp := sub('/\\d$', '', q)]
nr <- dt[, .(pair = length(unique(q))), keyby = qp]
setkey(dt, qp)
dt <- nr[dt]
head(dt)

,qp,pair,q,t,pos,len,align,specific,seq
1,H5JCKCCXX:1:10:149660:0,1,H5JCKCCXX:1:10:149660:0/1,B*35:52,242,49,49,0,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP
2,H5JCKCCXX:1:10:149660:0,1,H5JCKCCXX:1:10:149660:0/1,B*13:67,242,49,49,0,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP
3,H5JCKCCXX:1:10:149660:0,1,H5JCKCCXX:1:10:149660:0/1,B*15:163,242,49,49,0,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP
4,H5JCKCCXX:1:10:149660:0,1,H5JCKCCXX:1:10:149660:0/1,B*08:132,242,49,49,0,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP
5,H5JCKCCXX:1:10:149660:0,1,H5JCKCCXX:1:10:149660:0/1,B*58:70,242,49,49,0,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP
6,H5JCKCCXX:1:10:149660:0,1,H5JCKCCXX:1:10:149660:0/1,B*51:01,242,49,49,0,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP


In [109]:
nr <- dt[, .(pair2 = length(unique(q))), keyby = .(qp, t)]
setkey(dt, qp, t)
dt <- nr[dt]
head(dt)

,qp,t,pair2,pair,q,pos,len,align,specific,seq
1,H5JCKCCXX:1:10:149660:0,B*07:02,1,1,H5JCKCCXX:1:10:149660:0/1,242,49,49,0,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP
2,H5JCKCCXX:1:10:149660:0,B*07:03,1,1,H5JCKCCXX:1:10:149660:0/1,242,49,49,0,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP
3,H5JCKCCXX:1:10:149660:0,B*07:04,1,1,H5JCKCCXX:1:10:149660:0/1,242,49,49,0,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP
4,H5JCKCCXX:1:10:149660:0,B*07:05,1,1,H5JCKCCXX:1:10:149660:0/1,242,49,49,0,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP
5,H5JCKCCXX:1:10:149660:0,B*07:06,1,1,H5JCKCCXX:1:10:149660:0/1,242,49,49,0,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP
6,H5JCKCCXX:1:10:149660:0,B*07:07,1,1,H5JCKCCXX:1:10:149660:0/1,242,49,49,0,RDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLP


In [110]:
dt <- dt[pair == pair2]

In [111]:
library(IRanges)

In [112]:
dt <- dt[grepl('\\d$', t) & align / len > 0.8]
setkey(dt, t)
cov <- dt[, .(
    n0 = .N, 
    n1 = sum(specific), 
    cov0 = sum(width(reduce(IRanges(pos, width = len)))),
    cov1 = sum(width(reduce(IRanges(pos[specific == 1], width = len[specific == 1]))))
), keyby = t]

In [113]:
head(cov[order(-cov1)])

,t,n0,n1,cov0,cov1
1,B*51:01,76,45,297,297
2,B*51:193,76,45,297,297
3,B*51:96,73,42,295,295
4,B*51:130,63,35,294,294
5,B*51:163,62,34,293,293
6,B*51:187,75,44,294,292


In [114]:
head(cov[order(-n1)])

,t,n0,n1,cov0,cov1
1,B*13:02,96,57,274,273
2,B*13:38,74,55,272,265
3,B*13:70,85,54,267,267
4,B*13:67,83,52,257,257
5,B*13:82,88,52,264,263
6,B*13:15,86,50,262,261


In [115]:
pure <- dt[specific == 1]
nonpure <- dt[specific == 0]

In [116]:
nrow(pure)

[1] 26962

In [117]:
nrow(nonpure)

[1] 34892

In [118]:
good <- dt[align / len > 0.8]
nrow(good)

[1] 61854

In [141]:
tt <- good[, v := specific * 5 + 1]
mat <- dcast(good, q ~ t, value.var = 'v', fill = 0)

In [142]:
tt <- mat[, q := NULL]

In [143]:
tail(sort(colSums(mat)))

B*13:15 B*13:67 B*13:82 B*13:38 B*13:70 B*13:02 
    336     343     348     349     355     381

In [144]:
temp <- sort(colSums(mat))

In [151]:
tail(temp[grepl('B*51', names(temp))], 20)

B*51:142  B*51:50 B*51:157  B*51:30  B*51:78 B*51:159 B*51:175 B*51:161 
     265      265      266      267      268      271      271      272 
B*51:176  B*51:57 B*51:169 B*51:188 B*51:165  B*51:96 B*51:151  B*51:51 
     272      272      273      277      280      283      284      286 
B*51:164 B*51:187  B*51:01 B*51:193 
     292      295      301      301

In [146]:
mat[, sum(pmax(`B*13:02`, `B*51:30`))]

[1] 646

In [147]:
mat[, sum(pmax(`B*13:02`, `B*51:10`))]

[1] 490

In [148]:
mat[, sum(pmax(`B*13:82`, `B*51:30`))]

[1] 613

In [149]:
mat[, sum(pmax(`B*13:02`, `B*51:193`))]

[1] 613

In [150]:
mat[, sum(pmax(`B*13:02`, `B*51:01`))]

[1] 613

In [128]:
mat[, sum(pmax(`B*13:02`, `B*51:51`))]

[1] 116

In [129]:
mat[, sum(pmax(`B*13:02`, `B*51:165`))]

[1] 116

In [130]:
mat[, sum(pmax(`B*13:70`, `B*51:193`))]

[1] 111.5

In [131]:
mat[, sum(pmax(`B*13:70`, `B*51:01`))]

[1] 111.5

In [132]:
head(cov[grep('B*05:01', t)][order(-cov1)], 30)

,t,n0,n1,cov0,cov1


In [133]:
head(cov[grepl('51:', t)])

,t,n0,n1,cov0,cov1
1,B*51:01,76,45,297,297
2,B*51:02,70,39,282,276
3,B*51:03,69,39,281,276
4,B*51:04,66,35,291,291
5,B*51:05,64,36,266,266
6,B*51:06,64,33,267,267


In [55]:
nrow(mat)

[1] 151

In [100]:
head(cov)

,t,n0,n1,cov0,cov1
1,B*07:02,43,16,92,92
2,B*07:03,43,16,92,92
3,B*07:04,43,16,92,92
4,B*07:05,43,16,92,92
5,B*07:06,43,16,92,92
6,B*07:07,43,16,92,92
